# RNA-Seq Analysis Training Demo (Snakemake)

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial uses a popular workflow manager called 'snakemake'. More information on snakemake can be found <a href="https://snakemake.readthedocs.io/en/stable/">here</a>. Running the code in this tutorial will take approximately 12 minutes.

![RNA-Seq workflow](images/rnaseq-workflow.png)

## Learning Objectives

* **Understanding Snakemake configuration:** Learners will examine Snakemake configuration files (YAML) and understand their role in defining the workflow.

* **Running a Snakemake workflow:** Participants will execute a pre-written Snakemake workflow for RNA-Seq analysis, encompassing steps like read trimming, quality control, mapping, and read counting.  They learn to interpret and utilize a Snakefile.

* **Interpreting RNA-Seq results:**  Learners will learn how to access and interpret the output of the Snakemake workflow, including identifying highly expressed genes and examining the expression levels of specific genes of interest.  This involves using command-line tools like `sort`, `head`, and `grep`.

* **Navigating a modular workflow:** The notebook introduces the concept of modularity in bioinformatics analysis by showcasing additional related notebooks (workflows)  covering different aspects of RNA-Seq analysis (e.g., extended tutorials, differential gene expression analysis using R).  This emphasizes the organization and scalability of workflows.

## Prerequisites

This Jupyter Notebook performs RNA-Seq analysis using Snakemake, relying on several external tools and data sources. Here's a breakdown of the prerequisites:

**APIs:**

* **aws s3:**  The Amazon S3 Storag tool `aws s3` is used extensively to download data from an Amazon S3 bucket.  This implicitly requires the Amazon S3 API to be enabled.

**Software and Dependencies:**

* **Snakemake:** This workflow manager orchestrates the entire RNA-Seq pipeline.
* **Bioconda Channels:** The notebook uses bioconda channels to install bioinformatic tools.
* **Other tools (installed via bioconda):** The Snakemake workflow uses multiple bioinformatic tools (like Trimmomatic and Salmon) that will be installed through the specified conda environments. The notebook doesn't explicitly list all the tools used within the Snakemake workflow. This is specified within the `envs/*.yaml` files.
* **Sufficient disk space:** The workflow requires sufficient space to store the downloaded data, intermediate files, and results. The amount of space will depend on the size of the datasets processed.
* **Multiple CPU cores:** The notebook uses multiple cores (`--cores $CORES`) to speed up the analysis with snakemake. The number of cores utilized is determined by `nproc`.


## Get Started

**If you already completed Tutorial 1, you should be able to skip to Step 6: Snakemake Configs**

### STEP 1: Install the tools

In [ ]:
! conda install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc salmon

### STEP 2: Create directories that will be used in our analysis

In [ ]:
!echo $PWD
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/raw_fastqSub
!mkdir -p data/trimmed
!mkdir -p data/reference
!mkdir -p data/fastqc
!mkdir -p envs

In [ ]:
numthreads=!nproc
numthreadsint = int(numthreads[0])
%env CORES = $numthreadsint
#!echo ${CORES}

### STEP 3: Copy FASTQ Files
In order for this tutorial to run quickly, we will only analyze 50,000 reads from a sample from both sample groupsinstead of analyzing all the reads from all six samples. These files have been posted on an Amazon S3 Bucket that we made publicly accessible.

In [ ]:
! aws s3 cp --recursive s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/raw_fastqSub data/raw_fastq

### STEP 4: Copy reference transcriptome files that will be used by Salmon
Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

In [ ]:
! aws s3 cp s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/reference/M_chelonae_transcripts.fasta data/reference/M_chelonae_transcripts.fasta
! aws s3 cp s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/data/reference/decoys.txt data/reference/decoys.txt

### STEP 5: Copy data file for Trimmomatic

In [ ]:
! aws s3 cp s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config/TruSeq3-PE.fa .

Next download config files for our snakemake environment, as well as data files which we will analyze.

### STEP 6: Copy config files for Snakemake


In [ ]:
# Copy config and data files
! mkdir envs
! aws s3 cp s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/config.yaml .
! aws s3 cp s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/snakefile .
! aws s3 cp --recursive s3://nigms-sandbox/me-inbre-rnaseq-pipelinev2/envs/ envs/

#### Explanation of config files

Snakemake is unique in that it uses config files to manage workflows in the form of 'yaml' files, as well as a 'snakefile'.

Below is a brief example of some of the yaml config files:

In [ ]:
!printf "The config.yaml file contains our sample names:\n\n Config.yaml\n"
!cat config.yaml
!printf "\n\nThe env folder contains information pertaining to packages to be used in the environment, \nas well as their version, for example, here is the 'envs/fastqc.yaml' file:\n\n Fastqc.Yaml\n"
!cat envs/fastqc.yaml

### STEP 7: Install Snakemake

In [ ]:
! conda create -c conda-forge -c bioconda -n snakemake snakemake -y;

### STEP 8: Run Snakemake on our snakefile

Aside from the .yaml config files which information about software, dependencies, and versions -- snakemake uses a snakefile which contains information about a workflow.

This can be a powerful tool as it allows one to operate and think in terms of workflows instead of individual steps. 

Feel free to open the snakefile to look at it further. It is composed of 'rules' we have created.

Snakefiles work largely based on inputs. For a given input, there is an associated 'rule' that runs.

Snakefiles may take a while to get the idea of what's going on, but in simplest terms here we take an input of .fastq files, and based on the snakefile rules we created, those fastq files are run through the entire workflow of tutorial one.


In [ ]:
!ls data/raw_fastqSub

In [ ]:
! conda config --set channel_priority strict

In [ ]:
# Calling snakemake directly from Anaconda3 to avoid "conda activate snakemake" command
! /home/ec2-user/anaconda3/envs/snakemake/bin/snakemake --cores $CORES --forceall   

### STEP 9: Report the top 10 most highly expressed genes in the samples.

Top 10 most highly expressed genes in the wild-type sample. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`


In [ ]:
!sort -nrk 5,5 data/quants/SRR13349122_quant/quant.sf | head -10

Top 10 most highly expressed genes in the double lysogen sample.


In [ ]:
!sort -nrk 5,5 data/quants/SRR13349128_quant/quant.sf | head -10

### STEP 10: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
!grep 'BB28_RS16545' data/quants/SRR13349122_quant/quant.sf

Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [ ]:
!grep 'BB28_RS16545' data/quants/SRR13349128_quant/quant.sf

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as the standard short or extended tutorials.


[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://youtu.be/ChGfBR4do_Y>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)

## Conclusion

This Jupyter Notebook provided a comprehensive demonstration of RNA-Seq data analysis using Snakemake, a powerful workflow management system.  The tutorial efficiently processed a prokaryotic RNA-Seq dataset, covering key steps from read trimming and quality control using Trimmomatic and FastQC to read mapping and quantification of gene expression with Salmon.  The use of Snakemake streamlined the workflow, enabling reproducible and manageable execution of the entire pipeline.  The notebook successfully generated gene expression counts, allowing for downstream analyses like identifying differentially expressed genes (as shown by examples in the notebook) and further exploration using provided links to additional notebooks, including a DEG analysis workflow (Tutorial 3) and an extended version of the current Snakemake workflow focusing on scalability and broader data handling. This tutorial serves as a valuable resource for understanding and implementing RNA-Seq analysis using a robust and reproducible approach.  The clear instructions and readily available supplementary materials make it easily adaptable for both novice and experienced users.

## Clean Up

Remember to move to the next notebook or shut down your instance if you are finished.